<a href="https://colab.research.google.com/github/gurkaur2103/Resume-Bias-Generator/blob/main/BIAS_DETECTOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WELCOME TO THIS COLLAB NOTEBOOK
# RESUME BIAS DETECTOR

Install Streamlit


In [ ]:
!pip install streamlit -q
!pip install --upgrade streamlit watchdog

!wget -q -O - ipv4.icanhazip.com

34.141.207.86
34.141.207.86


In [ ]:
! streamlit run Bias_Detector.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.141.207.86:8501

⠼⠴your url is: https://nine-bees-lay.loca.lt
2025-02-17 14:54:55.931 Uncaught exception GET /_stcore/stream (127.0.0.1)
HTTPServerRequest(protocol='http', host='nine-bees-lay.loca.lt', method='GET', uri='/_stcore/stream', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/tornado/websocket.py", line 938, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/server/browser_websocket_handler.py", line 176, in open
    self._session_id = self._runtime.connect_session(
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packa

Installing the Necessary Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
import base64

Setting up the Streamlit App

In [ ]:
st.set_page_config(page_title="Hiring Bias Analysis", layout="wide")

st.title("Automated Hiring Bias Analysis Tool")

2025-02-17 13:43:54.747 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-17 13:43:54.749 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-17 13:43:54.929 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-02-17 13:43:54.931 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

Function to Generate NLP Explaination

In [ ]:
def generate_nlp_explanation(gender_selection_rates, race_selection_rates, job_factors_gender, job_factors_race, di_gender, di_race):
    explanation = "\nAutomated Hiring Bias Report\n"
    explanation += "---------------------------------------------------\n"

    explanation += "\nGender Bias Analysis:\n"
    explanation += f"- Women receive callbacks at {gender_selection_rates.get('female', 0) * 100:.2f}%, while men receive {gender_selection_rates.get('male', 0) * 100:.2f}%.\n"
    if di_gender < 0.8:
        explanation += "There is a significant gender bias in hiring. Women are less likely to be selected despite similar qualifications.\n"
    else:
        explanation += "No significant gender bias detected. The hiring process appears fair in terms of gender selection.\n"

    explanation += "\nRace Bias Analysis:\n"
    explanation += f"- White candidates receive callbacks at {race_selection_rates.get('white', 0) * 100:.2f}%, while Black candidates receive {race_selection_rates.get('black', 0) * 100:.2f}%.\n"
    if di_race < 0.8:
        explanation += "Racial bias is present in hiring. Black candidates are receiving significantly fewer callbacks compared to White candidates.\n"
    else:
        explanation += "No strong racial bias detected in hiring decisions.\n"

    explanation += "\nImpact of Job Qualifications on Hiring\n"

    for factor in job_factors_gender.columns:
        female_avg = job_factors_gender.loc['female', factor] if 'female' in job_factors_gender.index else 0
        male_avg = job_factors_gender.loc['male', factor] if 'male' in job_factors_gender.index else 0
        difference = female_avg - male_avg
        percentage_diff = (difference / max(male_avg, 0.001)) * 100

        if abs(percentage_diff) > 10:
            if difference > 0:
                explanation += f"- {factor.replace('_', ' ').title()}: Women score {percentage_diff:.1f}% higher than men.\n"
            else:
                explanation += f"- {factor.replace('_', ' ').title()}: Men score {abs(percentage_diff):.1f}% higher than women.\n"

    for factor in job_factors_race.columns:
        black_avg = job_factors_race.loc['black', factor] if 'black' in job_factors_race.index else 0
        white_avg = job_factors_race.loc['white', factor] if 'white' in job_factors_race.index else 0
        difference = black_avg - white_avg
        percentage_diff = (difference / max(white_avg, 0.001)) * 100

        if abs(percentage_diff) > 10:
            if difference > 0:
                explanation += f"- {factor.replace('_', ' ').title()}: Black candidates score {percentage_diff:.1f}% higher than White candidates.\n"
            else:
                explanation += f"- {factor.replace('_', ' ').title()}: White candidates score {abs(percentage_diff):.1f}% higher than Black candidates.\n"

    explanation += "\nFinal Verdict:\n"
    if di_gender < 0.8 or di_race < 0.8:
        explanation += "The hiring process appears biased. Certain groups are facing disadvantages despite their qualifications. Consider reviewing selection criteria to ensure fairness.\n"
    else:
        explanation += "No major bias detected. The hiring process appears fair.\n"

    return explanation


Function to Detect Bias

In [ ]:
def detect_bias(df):
    df['received_callback'] = df['received_callback'].astype(int)

    df['gender'] = df['gender'].str.lower().str.strip().replace({'m': 'male', 'f': 'female'})
    df = df[df['gender'].isin(['male', 'female'])]

    resume_mapping = {'low': 0, 'medium': 1, 'high': 2}
    df['resume_quality'] = df['resume_quality'].str.lower().str.strip().map(resume_mapping).fillna(1)

    job_factors = ['job_req_education', 'job_req_min_experience', 'job_req_computer',
                   'college_degree', 'years_experience', 'computer_skills', 'resume_quality']
    df[job_factors] = df[job_factors].apply(pd.to_numeric, errors='coerce').fillna(0)

    gender_selection_rates = df.groupby('gender')['received_callback'].mean()
    race_selection_rates = df.groupby('race')['received_callback'].mean()

    job_factors_gender = df[df['received_callback'] == 1].groupby('gender')[job_factors].mean()
    job_factors_race = df[df['received_callback'] == 1].groupby('race')[job_factors].mean()

    di_gender = np.nan if gender_selection_rates.empty else gender_selection_rates.min() / gender_selection_rates.max()
    di_race = np.nan if race_selection_rates.empty else race_selection_rates.min() / race_selection_rates.max()

    explanation = generate_nlp_explanation(gender_selection_rates, race_selection_rates, job_factors_gender, job_factors_race, di_gender, di_race)

    return explanation

Function to Download the Bias Report

In [ ]:
def download_report(report_text, filename="bias_report.txt"):
  """Generates a link allowing the data in a given panda dataframe to be downloaded
  in:  dataframe
  out: href string
  """
  b64 = base64.b64encode(report_text.encode()).decode()  # some strings <-> bytes conversions necessary here
  href = f'<a href="data:file/txt;base64,{b64}" download="{filename}">Download Report</a>'
  return href

Function to Plot Visualizations

In [ ]:
def plot_visuals(df):
    st.subheader("Visualizations")

    # Gender vs Callback Rate
    st.write("### Selection Rate by Gender")
    fig, ax = plt.subplots()
    sns.barplot(x=df['gender'], y=df['received_callback'], errorbar=None, ax=ax)
    ax.set_ylabel('Callback Rate')
    ax.set_xlabel('Gender')
    st.pyplot(fig)

    # Race vs Callback Rate
    st.write("### Selection Rate by Race")
    fig, ax = plt.subplots()
    sns.barplot(x=df['race'], y=df['received_callback'], errorbar=None, ax=ax)
    ax.set_ylabel('Callback Rate')
    ax.set_xlabel('Race')
    st.pyplot(fig)

    # Correlation Heatmap
    st.write("### Feature Correlation with Callback Rate")
    fig, ax = plt.subplots(figsize=(10, 6))
    correlation = df.corr()
    sns.heatmap(correlation, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5, ax=ax)
    st.pyplot(fig)

Main(Driver) Function for the Streamlit app

In [ ]:
def main():
    st.sidebar.header("Upload or Enter Data")
    upload_option = st.sidebar.radio("Choose an option:", ("Upload CSV", "Enter Data Manually"))

    if upload_option == "Upload CSV":
        uploaded_file = st.sidebar.file_uploader("Upload your CSV file", type=["csv"])
        if uploaded_file is not None:
            df = pd.read_csv(uploaded_file)
        else:
            st.warning("Please upload a CSV file.")
            return
    else:
        st.sidebar.write("Enter data manually:")
        manual_data = st.sidebar.text_area("Paste your data here (CSV format):")
        if manual_data:
            df = pd.read_csv(pd.compat.StringIO(manual_data))
        else:
            st.warning("Please enter data manually.")
            return

    if df is not None:
        st.write("### Data Preview")
        st.dataframe(df.head())

        explanation = detect_bias(df)
        st.write("### Bias Analysis Report")
        st.text(explanation)

        # Download button
        st.markdown(download_report(explanation), unsafe_allow_html=True)

        plot_visuals(df)

Run the App

In [ ]:
if __name__ == "__main__":
    main()

2025-02-17 13:47:50.453 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-17 13:47:50.455 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-17 13:47:50.456 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-17 13:47:50.457 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-17 13:47:50.459 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-17 13:47:50.461 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-17 13:47:50.463 Session state does not function when running a script without `streamlit run`
2025-02-17 13:47:50.465 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-17 13:47